In [1]:
import pandas as pd

In [2]:
game = pd.read_csv('C:/myprojects/KPL-Project/data/Seasons/2014/rounds.csv')
table = pd.read_csv('C:/myprojects/KPL-Project/data/Seasons/2014/table.csv')

In [3]:

game['Date'] = pd.to_datetime(game['Date'])

In [19]:
table.info()
game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   TeamName    16 non-null     object
 1   NameCode    16 non-null     object
 2   ShortName   16 non-null     object
 3   TeamID      16 non-null     int64 
 4   TeamColors  16 non-null     object
 5   Position    16 non-null     int64 
 6   Played      16 non-null     int64 
 7   Won         16 non-null     int64 
 8   Drawn       16 non-null     int64 
 9   Lost        16 non-null     int64 
 10  GF          16 non-null     int64 
 11  GA          16 non-null     int64 
 12  Points      16 non-null     int64 
dtypes: int64(9), object(4)
memory usage: 1.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Tournament        254 non-nul

In [21]:
teamdim = table[['TeamID', 'TeamName', 'NameCode', 'TeamColors']].drop_duplicates().reset_index(drop=True)
teamdim.set_index('TeamID')


,TeamName,NameCode,TeamColors
TeamID,,,
134306,Gor Mahia FC,GOR,"['#008000', '#000000']"
138488,Sofapaka,SOF,"['#ffffff', '#0000ff']"
138498,Ulinzi Stars,ULS,"['#ff0000', '#ffffff']"
138496,Tusker,TUS,"['#ffdf00', '#000000']"
138476,Chemelil Sugar,CSU,"['#ffff00', '#000000']"
76433,AFC Leopards SC,LDS,"['#ffffff', '#0000ff']"
138484,Muhoroni Youth FC,MUY,"['#374df5', '#374df5']"
138490,Sony Sugar,SSU,"['#008633', '#ffffff']"
138492,Thika United,TUN,"['#374df5', '#374df5']"


In [22]:
seasondim = game[['SeasonID', 'Season']].drop_duplicates().reset_index(drop=True)
seasondim.set_index('SeasonID')

KeyError: "None of [Index(['SeasonID', 'SeasonName'], dtype='object')] are in the [columns]"

In [20]:
facttable = game.merge(teamdim, on='HomeTeamID')\
                .merge(teamdim, on='AwayTeamID')\
                .merge(seasondim, on='SeasonID')\
                [['Round']]
facttable

KeyError: 'HomeTeamID'